In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.3 MB/s eta 0:00:00


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00


In [4]:
import torch
from transformers import LlamaTokenizer, AutoModelForVision2Seq, BlipImageProcessor
from PIL import Image
import requests

In [5]:
model = AutoModelForVision2Seq.from_pretrained("stabilityai/japanese-instructblip-alpha", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-instructblip-alpha:
- configuration_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-instructblip-alpha:
- configuration_japanese_instructblip_alpha.py
- configuration_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-instructblip-alpha:
- modeling_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-instructblip-alpha:
- modeling_japanese_instructblip_alpha.py
- modeling_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
processor = BlipImageProcessor.from_pretrained("stabilityai/japanese-instructblip-alpha", device=device)

### Signateのデータをダウンロード

In [8]:
!pip install signate

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=47dab067859001874708df688c94251cdd6dab440fffd13488fa48c0633cc970
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [18]:
!cp /content/drive/MyDrive/MyStudy/MySIGNATE/signate.json /root/.signate/

In [19]:
!signate download --competition-id=1106

dataset_definition.md

train.csv

sample_submit.csv

test.zip

train.zip


Download completed.


In [20]:
%%capture
!unzip train.zip

In [21]:
%%capture
!unzip test.zip

### 後続の小さいNNを学習

In [22]:
import os
import os.path as osp

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

In [23]:
class ImgDataset(Dataset):
    def __init__(self, img_dir, img_paths, labels, transform=None):
        self.img_dir = img_dir
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = osp.join(self.img_dir, self.img_paths[idx])
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        image = processor(images=img, return_tensors='pt')

        return image.pixel_values[0], self.labels[idx]

In [24]:
from torchvision import transforms

trans = transforms.Compose([
    # transforms.ToTensor(),
    transforms.AutoAugment(),
    transforms.RandomApply(
        nn.ModuleList([
            # transforms.ColorJitter(),
            transforms.GaussianBlur(kernel_size=11),
        ]),
        p=0.2
    ),
    # transforms.RandomRotation((-180, 180)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])

In [25]:
train_img_dir = "./train"
train_df = pd.read_csv("train.csv")

train, valid = train_test_split(train_df, test_size=0.2, stratify=train_df["label"], random_state=42, shuffle=True)
train_dataset = ImgDataset(img_dir="./train", img_paths=train["image_name"].values, labels=train["label"].values, transform=trans)
valid_dataset = ImgDataset(img_dir="./train", img_paths=valid["image_name"].values, labels=valid["label"].values)

In [26]:
batch_size = 32

In [27]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [28]:
class wrapperModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.vision_model = model.vision_model
        self.classifier_head = nn.Linear(1408, 2)

    def forward(self, x):
        out = self.vision_model(x).pooler_output
        y = self.classifier_head(out)

        return y

In [29]:
wrapper_model = wrapperModel()
wrapper_model.to(device)

wrapperModel(
  (vision_model): InstructBlipVisionModel(
    (embeddings): InstructBlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): InstructBlipEncoder(
      (layers): ModuleList(
        (0-38): 39 x InstructBlipEncoderLayer(
          (self_attn): InstructBlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): InstructBlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (pos

In [30]:
for name, param in wrapper_model.named_parameters():
    if "vision_model" in name:
        param.requires_grad = False

In [31]:
EPOCH_NUM = 50
optimizer = Adam(wrapper_model.parameters(), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCH_NUM, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()

In [32]:
writer = SummaryWriter(log_dir="./log_full_train")

In [33]:
torch.cuda.empty_cache()

In [36]:
prev_loss = float("inf")
best_loss = float("inf")
pbar = tqdm(range(EPOCH_NUM))
l2_coef = 1e-3
for epoch in pbar:

    wrapper_model.train()
    for iter_num, (X, y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        with torch.autocast("cuda"):
            pred_y = wrapper_model(X)

            l2_loss = 0.0
            for name, param in wrapper_model.named_parameters():
                if 'classifier_head' in name:
                    l2_loss += torch.norm(param)**2
            loss = criterion(pred_y, y) + l2_coef*l2_loss

        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch/Iter {epoch}/{iter_num}: loss{loss.item():.4f} / {prev_loss:.4f}")
        writer.add_scalar("train/loss", loss.item(), epoch*len(train_dataloader)+iter_num)
    scheduler.step()

    val_loss = 0
    wrapper_model.eval()
    for iter_num, (X, y) in enumerate(valid_dataloader):
        X = X.to(device)
        y = y.to(device)

        with torch.no_grad(), torch.autocast("cuda"):
            pred_y = wrapper_model(X)
            loss = criterion(pred_y, y)
            val_loss += loss.item()

        writer.add_scalar("train/loss", loss.item(), epoch*len(valid_dataloader)+iter_num)

    prev_loss = val_loss / len(valid_dataloader)
    if best_loss > prev_loss:
        best_loss = prev_loss
        torch.save(wrapper_model.state_dict(), "/content/drive/MyDrive/MyStudy/MySIGNATE/package-classification-comp/models/blip_and_NN/classifier_head_full.pt")
    print(prev_loss)

Epoch/Iter 0/54: loss0.4976 / inf:   2%|▏         | 1/50 [01:07<55:14, 67.65s/it]

0.4066068615232195


Epoch/Iter 1/54: loss0.5926 / 0.4066:   4%|▍         | 2/50 [02:18<55:23, 69.24s/it]

0.4064413160085678


Epoch/Iter 2/54: loss0.5668 / 0.4064:   6%|▌         | 3/50 [03:14<49:49, 63.61s/it]

...

0.40350911234106335


Epoch/Iter 32/54: loss0.3853 / 0.4035:  66%|██████▌   | 33/50 [33:33<17:10, 60.62s/it]

0.3833236257944788


Epoch/Iter 33/46: loss0.3735 / 0.3833:  66%|██████▌   | 33/50 [34:15<17:38, 62.28s/it]


KeyboardInterrupt: ignored

In [37]:
%load_ext tensorboard

In [43]:
!tensorboard --logdir /content/log_full_train

2023-09-23 10:20:09.524987: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
wrapper_model.load_state_dict(torch.load('/content/drive/MyDrive/MyStudy'))

In [ ]:
preds = torch.empty((len(valid_dataset), 2))
wrapper_model.eval()
for iter_num, (X, y) in enumerate(valid_dataloader):
    X = X.to(device)
    y = y.to(device)

    with torch.no_grad(), torch.autocast("cuda"):
        pred_y = wrapper_model(X)
        preds[iter_num*batch_size:(iter_num+1)*batch_size] = pred_y.detach()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
fpr, tpr, thresholds = roc_curve(valid["label"], preds[:, 1].numpy())
roc_auc = roc_auc_score(valid["label"], preds[:, 1].numpy())

In [ ]:
plt.plot(fpr, tpr, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()